# Sign Language Classification

Даны  картинки с символами языка жестов ASL (Americal Sign Language). Необходимо построить модель машинного обучения, которая смогла бы правильно распознавать символы языка жестов и классифицировать их в определенные группы. 

![](https://user-images.githubusercontent.com/34737471/55292500-52acb580-5409-11e9-9a78-7cdc0d8da8b6.png)

## 1. Первичный анализ данных

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
train_df = pd.read_csv('../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv')
test_df = pd.read_csv('../input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv')

In [4]:
train_df.head()

In [5]:
train_df.shape, test_df.shape

In [6]:
y_train = train_df['label']
y_test = test_df['label']

X_train = train_df.drop(columns='label')
X_test = test_df.drop(columns='label')

In [7]:
def show_images(images,labels):
    images = images.values.reshape(-1,28,28,1)
    unique_labels = y_train.unique()
    fig,ax = plt.subplots(2,5)
    fig.set_size_inches(10, 6)
    k =0
    for i in range(2):
        for j in range(5):
            ax[i,j].imshow(images[k] , cmap='gray')
            ax[i,j].set_title(str(unique_labels[y_train[k]]))
            k = k+1;
    plt.tight_layout()

In [8]:
show_images(X_train, y_train)

Распределение классов в trainset

In [9]:
plt.figure(figsize=(15,6))
sns.set_style("darkgrid")
sns.countplot(x=y_train)
plt.show()

Распределение классов в testset

In [10]:
plt.figure(figsize=(15,6))
sns.set_style("darkgrid")
sns.countplot(x=y_test)
plt.show()

## 2. Построение простых методов многоклассовой классификации

Использованные модели:

* Линейные модели
* Решающие деревья
* Байесовские методы

In [11]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import ExtraTreeClassifier, DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier
from xgboost.sklearn import XGBClassifier 
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [12]:
svc = SVC(random_state=5)
svc.fit(X_train, y_train)
print('='*25)
print('SVC')
print(f'accuracy of train set: {svc.score(X_train, y_train)}')
print(f'accuracy of test set: {svc.score(X_test, y_test)}')

In [13]:
lsvc = LinearSVC()
lsvc.fit(X_train, y_train)
print('='*25)
print('LinearSVC')
print(f'accuracy of train set: {lsvc.score(X_train, y_train)}')
print(f'accuracy of test set: {lsvc.score(X_test, y_test)}')

In [14]:
sgd = SGDClassifier()
sgd.fit(X_train, y_train)
print('='*25)
print('SGD Classifier')
print(f'accuracy of train set: {sgd.score(X_train, y_train)}')
print(f'accuracy of test set: {sgd.score(X_test, y_test)}')

In [15]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
print('='*25)
print('KNeighborsClassifier')
print(f'accuracy of train set: {knn.score(X_train, y_train)}')
print(f'accuracy of test set: {knn.score(X_test, y_test)}')

In [16]:
rf = RandomForestClassifier(random_state=5)
rf.fit(X_train, y_train)
print('='*25)
print('RandomForestClassifier')
print(f'accuracy of train set: {rf.score(X_train, y_train)}')
print(f'accuracy of test set: {rf.score(X_test, y_test)}')

In [17]:
decisiontree = DecisionTreeClassifier(max_depth=50, random_state=5)
decisiontree.fit(X_train, y_train)
print('='*25)
print('DecisionTreeClassifier')
print(f'accuracy of train set: {decisiontree.score(X_train, y_train)}')
print(f'accuracy of test set: {decisiontree.score(X_test, y_test)}')

In [18]:
bagging = BaggingClassifier(ExtraTreeClassifier(random_state=5))
bagging.fit(X_train, y_train)
print('='*25)
print('BaggingClassifier')
print(f'accuracy of train set: {bagging.score(X_train, y_train)}')
print(f'accuracy of test set: {bagging.score(X_test, y_test)}')

In [19]:
lgbm = LGBMClassifier()
lgbm.fit(X_train, y_train)
print('='*25)
print('LGBM Classifier')
print(f'accuracy of train set: {lgbm.score(X_train, y_train)}')
print(f'accuracy of test set: {lgbm.score(X_test, y_test)}')

In [20]:
catboost = CatBoostClassifier(learning_rate=0.3, max_depth=10, n_estimators=25)
catboost.fit(X_train, y_train)
print('='*25)
print('CatBoost Classifier')
print(f'accuracy of train set: {catboost.score(X_train, y_train)}')
print(f'accuracy of test set: {catboost.score(X_test, y_test)}')

In [21]:
%%time
estimators = [
    ("svc", svc), 
    ("knn", knn),
    ("rf", rf),
    ("extratree", bagging),
    ("lgbm", lgbm)]

vote = VotingClassifier(estimators=estimators)
vote.fit(X_train, y_train)
print('='*25)
print('VotingClassifier')
print(f'accuracy of train set: {vote.score(X_train, y_train)}')
print(f'accuracy of test set: {vote.score(X_test, y_test)}')

In [22]:
y_pred = vote.predict(X_test)

In [23]:
print(classification_report(y_test, y_pred))

In [24]:
plt.figure(figsize=(15,6))
cf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(cf_matrix, annot=True)
plt.show()

## 3. Использование методов понижения размерности

In [12]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [33]:
y_train = train_df['label']
y_test = test_df['label']

X_train = train_df.drop(columns='label')
X_test = test_df.drop(columns='label')

In [34]:
X = pd.concat([X_train, X_test])
y = pd.concat([y_train, y_test])

In [35]:
%%time
only_pca_model = PCA()
only_pca_model.n_components = 200
only_pca_data = only_pca_model.fit_transform(X/255)
only_pca_data = np.vstack((only_pca_data.T)).T

pca_tsne_model = TSNE(n_components=2, random_state=0, perplexity=70, n_iter=1000)
pca_tsne_data = pca_tsne_model.fit_transform(only_pca_data)

pca_tsne_data = np.vstack((pca_tsne_data.T, y)).T
pca_tsne_train = pd.DataFrame(pca_tsne_data, columns = ('PC 1', 'PC 2', 'label'))
pca_tsne_train.head()

In [36]:
sns.FacetGrid(pca_tsne_train, hue='label', height=8).map(plt.scatter, 'PC 1', "PC 2").add_legend()
plt.show()

In [37]:
X_pca_train = pca_tsne_train.drop(columns=['label'])[:27455]
y_pca_train = pca_tsne_train['label'][:27455]

X_pca_test = pca_tsne_train.drop(columns=['label'])[27455:].reset_index(drop=True)
y_pca_test = pca_tsne_train['label'][27455:].reset_index(drop=True)

In [38]:
X_pca_train = X_pca_train.join(X_train)
X_pca_test = X_pca_test.join(X_test)

In [39]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_pca_train, y_pca_train)
print('='*25)
print('KNeighborsClassifier')
print(f'accuracy of train set: {knn.score(X_pca_train, y_pca_train)}')
print(f'accuracy of test set: {knn.score(X_pca_test, y_pca_test)}')

In [40]:
rf = RandomForestClassifier(random_state=5)
rf.fit(X_pca_train, y_pca_train)
print('='*25)
print('RandomForestClassifier')
print(f'accuracy of train set: {rf.score(X_pca_train, y_pca_train)}')
print(f'accuracy of test set: {rf.score(X_pca_test, y_pca_test)}')

In [13]:
only_pca_model = PCA()
only_pca_model.n_components = 200
only_pca_data = only_pca_model.fit_transform(X_train/255)
only_pca_data = np.vstack((only_pca_data.T, y_train)).T

pca_tsne_model = TSNE(n_components =2, random_state =0, perplexity =50, n_iter=1000)
pca_tsne_data = pca_tsne_model.fit_transform(only_pca_data)

pca_tsne_data = np.vstack((pca_tsne_data.T, y_train)).T
pca_tsne_df = pd.DataFrame(pca_tsne_data, columns = ('PC 1', 'PC 2', 'label'))
pca_tsne_df.head()

In [15]:
sns.FacetGrid(pca_tsne_df, hue='label', height=8).map(plt.scatter, 'PC 1', "PC 2").add_legend()
plt.show()

## 3. Построение нейронных сетей

In [16]:
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

In [30]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.layer_1 = nn.Linear(input_size, hidden_size)
        self.layer_2 = nn.Linear(hidden_size, 128)
        self.layer_3 = nn.Linear(128, num_classes)
        
        self.relu = nn.ReLU()
        
    def forward(self, inputs):
        output_1 = self.relu(self.layer_1(inputs))
        output_2 = self.relu(self.layer_2(output_1))
        output = self.layer_3(output_2)
        
        return output

In [33]:
train_df = pd.read_csv('../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv')
test_df = pd.read_csv('../input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv')

In [34]:
train_labels = train_df['label'].values 
test_labels=test_df['label'].values
train_images = (train_df.iloc[:,1:].values).astype('float32')
test_images = (test_df.iloc[:,1:].values).astype('float32')

In [35]:
train_images = train_images.reshape(train_images.shape[0], 28, 28)
test_images = test_images.reshape(test_images.shape[0], 28, 28)

In [36]:
train_images_tensor = (torch.tensor(train_images)/255.0)
train_labels_tensor = (torch.tensor(train_labels))
train_tensor = TensorDataset(train_images_tensor, train_labels_tensor)

test_images_tensor = (torch.tensor(test_images)/255.0)
test_labels_tensor = (torch.tensor(test_labels))
test_tensor = TensorDataset(test_images_tensor, test_labels_tensor)

In [37]:
train_loader = DataLoader(train_tensor, batch_size=16, num_workers=2, shuffle=True)
test_loader = DataLoader(test_tensor, batch_size=16, num_workers=2, shuffle=False)

In [50]:
model = NeuralNet(28*28, 500, 25)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epoch = 50

In [51]:
torch.manual_seed(0)

total_step = len(train_loader)
for epoch in range(num_epoch):
    for i, (images, labels) in enumerate(train_loader):  
        images = images.reshape(-1, 28*28)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    
    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
           .format(epoch+1, num_epoch, i+1, total_step, loss.item()))

In [54]:
with torch.no_grad():
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(train_loader): 
        images = images.reshape(-1, 28*28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy: {} %'.format(100 * correct / total))

In [55]:
pred_lst = []
with torch.no_grad():
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(test_loader): 
        images = images.reshape(-1, 28*28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        pred_lst.extend(list(predicted))
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy: {} %'.format(100 * correct / total))